In [1]:
import shapely

In [2]:
import fiona

In [8]:
import csv
with open('some.csv', 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        print(row)

OrderedDict([('name', 'Chicago'), ('lat', '41.88'), ('lon', '-87.63')])
OrderedDict([('name', 'Kansas City'), ('lat', '39.101'), ('lon', '-94.584')])


In [16]:
import csv
from shapely.geometry import Point
with open('some.csv', 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        point = Point(float(row['lon']), float(row['lat']))
        print(row)
        print(point)

OrderedDict([('name', 'Chicago'), ('lat', '41.88'), ('lon', '-87.63')])
POINT (-87.63 41.88)
OrderedDict([('name', 'Kansas City'), ('lat', '39.101'), ('lon', '-94.584')])
POINT (-94.584 39.101)


In [11]:
import csv
from shapely.geometry import Point, mapping
from fiona import collection

schema = { 'geometry': 'Point', 'properties': { 'name': 'str' } }
with collection(
    "some.shp", "w", "ESRI Shapefile", schema) as output:
    with open('some.csv', 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            point = Point(float(row['lon']), float(row['lat']))
            output.write({
                'properties': {
                    'name': row['name']
                },
                'geometry': mapping(point)
            })

In [12]:
from shapely.geometry import mapping, shape
from fiona import collection

with collection("some.shp", "r") as input:
    # schema = input.schema.copy()
    schema = { 'geometry': 'Polygon', 'properties': { 'name': 'str' } }
    with collection(
        "some_buffer.shp", "w", "ESRI Shapefile", schema) as output:
        for point in input:
            output.write({
                'properties': {
                    'name': point['properties']['name']
                },
                'geometry': mapping(shape(point['geometry']).buffer(5.0))
            })

In [13]:
from shapely.geometry import mapping, shape
from shapely.ops import cascaded_union
from fiona import collection

with collection("some_buffer.shp", "r") as input:
    schema = input.schema.copy()
    with collection(
            "some_union.shp", "w", "ESRI Shapefile", schema) as output:
        shapes = []
        for f in input:
            shapes.append(shape(f['geometry']))
        merged = cascaded_union(shapes)
        output.write({
            'properties': {
                'name': 'Buffer Area'
                },
            'geometry': mapping(merged)
            })